In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Video Analysis with Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/video_analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fvideo-analysis%2Fvideo_analysis.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>       
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/video_analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/video-analysis/video_analysis.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner) |

## Getting Started


### Install Vertex AI SDK for Python

In [2]:
%pip install --upgrade --user -q google-cloud-aiplatform

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [3]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
# Define project information
PROJECT_ID = "qwiklabs-gcp-02-66bfa367acb3"  # @param {type:"string"}
LOCATION = "us-east4"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
# Please like share & subscribe to Techcps
# YouTube https://www.youtube.com/@techcps

print("Please like share & subscribe to Techcps https://www.youtube.com/@techcps")

### Import libraries


In [3]:
from IPython.display import Markdown, Video, display
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

### Load the Gemini 1.5 Pro model

Gemini 1.5 Pro (`gemini-1.5-pro`) is a multimodal model that supports multimodal prompts. You can include text, image(s), PDFs, audio, and video in your prompt requests and get text or code responses.

In [4]:
model = GenerativeModel("gemini-1.5-pro")

generation_config = GenerationConfig(temperature=1, top_p=0.95, max_output_tokens=8192)

safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

### Helper functions

In [5]:
def get_url_from_gcs(gcs_uri: str) -> str:
    # converts GCS uri to url for display.
    return gcs_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20"
    )

## Sentiment Analysis from Video and Audio

For this example, we will be analyzing a video of the first televised US presidential debate between John F. Kennedy and Richard Nixon. This debate is largely believed to have been critical in securing JFK's victory due to his calm and personable demeanor for the broadcast.

The video is an hour long, which is roughly equivalent to 1 Million Tokens for Gemini 1.5 Pro.

In [6]:
video_analysis_prompt = """You are an expert in politics and history. Provide a detailed analysis of the video including each speakers facial expressions and mood at key points."""

In [7]:
# Load file directly from Google Cloud Storage
video_uri = "gs://github-repo/video/KennedyNixon1960PresidentialDebate.mp4"

# Load contents
contents = [
    Part.from_uri(
        uri=video_uri,
        mime_type="video/mp4",
    ),
    video_analysis_prompt,
]

# Display the Video
display(Video(get_url_from_gcs(video_uri)))

Note: due to the length of the video, this will take a few minutes to complete. If you receive a 500 error, please wait a few moments and run the cell again.

In [8]:
# Send to Gemini
response = model.generate_content(contents, generation_config=generation_config)

# Display results
display(Markdown(response.text))

The video clip is of the first televised presidential debate between Richard Nixon and John F. Kennedy. The date of the debate is September 26, 1960. 

The first speaker is Howard K. Smith, an American broadcast journalist, who introduces the two candidates and sets out the rules of the debate. He speaks in a professional tone, as if giving a presentation. His face remains fairly neutral throughout his introduction.

At 0:46, Smith introduces Senator Kennedy and we see him get out of his chair and walk to the podium. His body language suggests he's eager to speak as he briskly walks to the podium. 

Kennedy begins speaking at 0:50. Kennedy quotes Abraham Lincoln and then draws a comparison between the political climate of Lincoln's time to the present day, stating that in the 1960's, "the question is whether the world will exist half-slave or half-free." He speaks slowly and steadily, and uses dramatic pauses for emphasis. He blinks frequently, but maintains a confident, almost defiant expression. 

At 1:20, Kennedy says, "I think it will depend in great measure upon what we do here in the United States," and taps his fingers on the podium for emphasis. 

Between 1:40 and 2:10, Kennedy becomes more passionate as he discusses the shortcomings he sees in the current American system, and what he believes needs to improve. He speaks faster, blinks more frequently, and makes a chopping motion with his hand when discussing the fact that “Mr. Khrushchev is in New York." His face is tight, and his brow is furrowed. 

Between 2:15 and 2:25, Kennedy transitions to speaking about the strengths of America and its people, and his facial expressions reflect this shift. His expression softens, and he appears more optimistic. 

By 2:30, Kennedy returns to discussing the shortcomings of the current administration and American systems. His expression becomes tight again, and his brow furrows as he lists social issues that he finds unsatisfactory. 

At 3:21, Kennedy states, “I'm not satisfied when we have over $9 billion worth of food, some of it rotting, even though there is a hungry world.”  He pauses briefly, and his face falls, momentarily betraying sadness at the situation. 

As he continues, he says, “I'm not satisfied when the Soviet Union is turning out twice as many scientists and engineers as we are.” At this point, his expression is tight with concern. 

At 6:59, Kennedy begins to transition back to a hopeful tone. He smiles faintly, saying, “The reason Franklin Roosevelt was a good neighbor in Latin America was because he was a good neighbor in the United States.”

At 7:10, Kennedy's smile widens, and he speaks passionately as he says, “I want us to recapture that image. I want people in Latin America, and Africa, and Asia to start to look to America, to see how we're doing things, to wonder what the President of the United States is doing." 

He concludes his remarks with a more neutral expression, again quoting Lincoln, “I think our generation of Americans has the same rendezvous. The question now is, can freedom be maintained under the most severe attack, attack, it has ever known? I think it can be.” 

At 8:00, the moderator introduces Nixon, who takes a drink of water and smiles confidently as he steps up to the podium.

Between 8:10 and 8:30, Nixon begins his remarks in a serious, measured tone. His expression is neutral. 

When Nixon says, “We are in a deadly competition, a competition not only with the men in the Kremlin, but the men in Peking,” his expression remains neutral, but he raises his eyebrows slightly.

At 8:47, Nixon begins to speak more passionately. He leans forward on the podium, his voice grows in volume, and he uses his hands to emphasize his points.

At 9:49, Nixon smiles as he quotes a “famous campaigner.” 

At 9:56, Nixon says, “Is the United States standing still? Is it true that this administration as Senator Kennedy has charged has been an administration of retreat, of defeat, of stagnation?”  His face hardens and his eyebrows rise as he poses these questions. 

At 10:10, Nixon says, “It’s not a question of which government does the most, it’s a question of which administration does the right thing.” He raises his eyebrows and furrows his brow. 

Between 10:40 and 11:00, Nixon continues to speak quickly and with a sense of urgency, but his expression softens as he discusses his party's record on social issues. 

At 11:10, Nixon says, “Let’s put it in terms that all of us can understand. We often hear growth in national product discussed, and in that respect may I say that when we compare the growth in this administration with that of the previous administration, then there was a total growth of 11% over seven years. In this administration there’s been a total growth of 19% over seven years.” His eyebrows are slightly raised as he states these figures. 

At 11:57, Nixon says, “We’ve had a higher tax load from the treasury for the farmer in the last few years, with the lowest farm income in many years.”  His expression hardens, and his brow furrows slightly.

As he moves on to discuss what he believes to be the correct strategy, his expression softens. 

At 12:26, Nixon says, “I believe that we have the secret for progress. We know the way to progress,” and taps on the podium for emphasis. 

By 12:49, he’s speaking faster again, and looks slightly agitated, as he says, “Now does this mean that his program is better than ours? Not at all.”  His face relaxes slightly when he pivots to discussing the goals the two parties have in common. 

Between 13:40 and 13:50, Nixon discusses poverty, unemployment, and the struggles of older Americans, his face shows compassion and concern. 

He concludes his remarks by stating, “We want higher teacher salaries. We need higher teacher salaries, but we also want our education to be free of federal control.”  His voice grows in volume when he says, "We need higher teacher salaries,”  and looks stern as he says, “free of federal control.”

At 16:15, after Nixon finishes speaking, Smith says, "Thank you Mr. Nixon." He then tells viewers, "That completes the opening statements and now the candidates will answer questions or comment upon one another's answers to questions put by correspondents of the networks, the correspondents." Throughout this, he’s speaking in a professional tone and his facial expressions are neutral.

At 16:26, Smith introduces the four network correspondents. All four men introduce themselves in a calm, even tone, and their expressions are neutral. 

At 16:39, Fleming, of ABC News, poses the first question to Senator Kennedy, and we see him rise from his chair and walk to the podium. 

At 16:54, Kennedy says, “The Vice President and I came to the Congress together in 1946. We both served on the Labor Committee. I’ve been there now for 14 years, the same period of time as he has, so that our experience in ah, government is comparable.”  He’s leaning slightly forward, speaking rapidly, but his expression is relaxed as he recounts his experience in government. 

At 17:26, Kennedy turns his attention to the Republican Party, and his expression becomes more serious. He says, "Mr. Nixon comes out of the Republican Party. He was nominated by it. And it is a fact that through most of these last 25 years the Republican leadership has opposed federal aid to education, medical care for the aged, development of the Tennessee Valley, development of our natural resources."  He emphasizes the word "opposed" with a firm nod, and his voice takes on a hard edge.  

Kennedy pauses, then says in a calmer tone, “I think Mr. Nixon is an effective leader of his party. I hope he would grant me the same. The question before us is, which point of view and which party do we want to lead the United States?”  His expression softens as he speaks these last few sentences. 

At 17:56, after Kennedy finishes speaking, the moderator asks Nixon if he wants to comment on Kennedy's answer. Nixon appears annoyed and shakes his head, curtly stating, "I have no comment." 

At 18:01,  the moderator, Novins, addresses Nixon, asking him about his eight years of experience and its value. Nixon answers the question with a sense of urgency, leaning forward and speaking with a firm, assertive tone. 

At 19:16, Nixon is asked by Novins to elaborate on which policies he helped shape during his time in the administration. Nixon smiles slightly as he answers, saying, “It would be rather difficult to cover them in eight in two and a half minutes. I would suggest that these proposals could be mentioned.”  He maintains a calm, confident expression as he lists some of his accomplishments.

At 19:36, Nixon is asked a question about his previous position on federal funding for schools and teacher salaries. Nixon admits that he "favored that" because the "local communities and your states are not going to meet the responsibility as much as they should."  He nods firmly, appears to be earnest, and looks directly into the camera. 

At 26:19, Vanocur quotes Eisenhower,  who stated that if given a week he "might" be able to think of a major idea Nixon has had that Eisenhower has adopted. He then says, “Now, that was a month ago sir, and the President hasn’t brought it up since. And I’m wondering sir if you can clarify which version is correct, the one put out by Republican campaign leaders or the one put up by President Eisenhower?”  Vanocur smiles slightly as he delivers the question, perhaps finding the situation amusing. 

Nixon says, “Well, I would just suggest Mr. Vanocur that if you know the President, that was probably a facetious remark.”  He chuckles, seemingly taking the President’s comment in good humor. 

The next question goes to Kennedy and asks him how serious he believes the "communist subversive activities in the United States today" are. Kennedy gives a solemn nod, blinking slowly as he answers, “Well, I think they’re serious. I think it’s a matter that we should continue to ah, give uh, great care and attention to.”

Nixon is asked if he’d like to comment, to which he simply nods, saying, “I agree with Senator Kennedy’s appraisal generally in this respect.” He continues speaking in a serious, urgent tone, and appears to be very focused. 

At 42:46, the moderator asks Kennedy, “Now, how do you feel that you’ll be able to get them in January if you weren’t able to get them in August?” The moderator’s tone is neutral. 

Kennedy glances to the side as he begins to answer,  saying, “Well, let’s take the bill, we did pass in the Senate, ah, to provide a $1.25 minimum wage.”  His facial expression is neutral, almost bored, but he picks up speed and looks directly at the camera as he continues, “It failed because the House did not pass it, and the House failed by 11 votes. And I might say that two-thirds of the Republicans in the House voted against a $1.25 minimum wage.” 

He closes his remarks by saying, "I don't want the federal government paying teachers' salaries directly, but if the money will go to the states, and the states can then determine whether the money would be spent for school construction or for teachers' salaries, in my opinion you protect the local authority over the school board and the school committees.” He makes a chopping motion with his hand as he emphasizes the words “local authority.”

At 47:51, after Nixon answers the previous question, Vanocur asks Nixon, “Now sir, isn’t it true that the building of more schools is a local matter for financing, uh were you claiming that the Eisenhower administration was responsible for the building of these schools or is it the local school districts that provide for them?” Vanocur looks directly at Nixon and delivers his question in a neutral tone. 

Nixon appears slightly agitated as he answers. He speaks quickly and leans toward the microphone as he lists examples of ways that the Eisenhower administration has supported local initiatives. 

At 48:49, he says, “I would say this,  that we, of course, expect to pick up some seats in both in the House and the Senate.” He smiles as he delivers this line, and appears to be momentarily pleased by the thought. 

At 51:02, Smith announces that it is time for the candidates’ closing statements. He does so in a neutral tone, and his facial expressions remain neutral. 

At 51:22, Nixon begins his closing statement, saying, "First of all, I think it is well to put in perspective where we really do stand with regard to the Soviet Union in this whole matter of growth." He stares directly at the camera as he speaks, and his expression is serious. 

At 52:15, Nixon begins to speak more quickly and intensely. He seems earnest and passionate as he makes his case, his eyebrows raising slightly as he criticizes the opposing party. 

The video ends with Nixon still speaking. 
